In [ ]:
# !pip install chromadb tiktoken

: 

In [11]:
from langchain_openai import OpenAIEmbeddings
# from langchain.vectorstores import Chroma

In [12]:
from langchain_chroma import Chroma

In [13]:
from langchain.schema import Document

In [14]:
# Create LangChain documents for IPL players

doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )

In [15]:
docs = [doc1, doc2, doc3, doc4, doc5]

In [ ]:
vector_store = Chroma(
    embedding_function=OpenAIEmbeddings(), # provide embedding function/model
    persist_directory='my_chroma_db', # loacation where embeddings will be saved
    collection_name='sample' # create a collection
)

### Chroma saves the database in a `.sqlite3` database

In [ ]:
# add documents
vector_store.add_documents(docs)


# every document is given a unique id by default

['4fbadedb-e4e1-4da1-87c3-800b8ad7ca3e',
 'c21aa93f-2f36-4bb6-a565-e2818f6a2bce',
 'c8bca5e8-3caf-43f6-8557-1a75623a9dc8',
 'c83ffd12-78ea-4d67-a9dd-a2cdba7b1c29',
 '3a657a6d-8814-4236-af6f-8a549be5123e']

In [ ]:
# view documents -> using .get
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['4fbadedb-e4e1-4da1-87c3-800b8ad7ca3e',
  'c21aa93f-2f36-4bb6-a565-e2818f6a2bce',
  'c8bca5e8-3caf-43f6-8557-1a75623a9dc8',
  'c83ffd12-78ea-4d67-a9dd-a2cdba7b1c29',
  '3a657a6d-8814-4236-af6f-8a549be5123e'],
 'embeddings': array([[-2.07734108e-03, -2.16356432e-03,  2.67739072e-02, ...,
         -1.70658119e-02, -3.68524459e-03,  1.35402400e-02],
        [-2.70681921e-03, -5.47363961e-05,  2.82030534e-02, ...,
         -1.50933377e-02,  5.93815418e-03, -1.16601437e-02],
        [ 7.83727213e-04, -4.76402277e-03,  1.23708826e-02, ...,
         -1.72387529e-02,  7.74397107e-04,  2.93532619e-03],
        [-2.71391869e-02,  8.87096301e-03,  2.69362777e-02, ...,
         -2.58583203e-02,  9.02314577e-03, -1.99992992e-02],
        [-1.81433018e-02,  1.27527108e-02,  3.47932205e-02, ...,
         -3.03654689e-02, -5.85236121e-03,  5.21374308e-03]],
       shape=(5, 1536)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his a

In [ ]:
# search documents
vector_store.similarity_search(
    query='Who among these are a bowler?',
    k=2 # how many similar vectors you want
)

[Document(id='c83ffd12-78ea-4d67-a9dd-a2cdba7b1c29', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
 Document(id='3a657a6d-8814-4236-af6f-8a549be5123e', metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.')]

In [20]:
# search with similarity score
vector_store.similarity_search_with_score(
    query='Who among these are a bowler?',
    k=2
)

[(Document(id='c83ffd12-78ea-4d67-a9dd-a2cdba7b1c29', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  0.35432544350624084),
 (Document(id='3a657a6d-8814-4236-af6f-8a549be5123e', metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  0.4086669683456421)]

In [21]:
# meta-data filtering
vector_store.similarity_search_with_score(
    query="",
    filter={"team": "Chennai Super Kings"}
)

[(Document(id='c8bca5e8-3caf-43f6-8557-1a75623a9dc8', metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  0.6488070487976074),
 (Document(id='3a657a6d-8814-4236-af6f-8a549be5123e', metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  0.6565988659858704)]

In [ ]:
# update existing documents
updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

vector_store.update_document(document_id='09a39dc6-3ba6-4ea7-927e-fdda591da5e4', document=updated_doc1)

In [23]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['4fbadedb-e4e1-4da1-87c3-800b8ad7ca3e',
  'c21aa93f-2f36-4bb6-a565-e2818f6a2bce',
  'c8bca5e8-3caf-43f6-8557-1a75623a9dc8',
  'c83ffd12-78ea-4d67-a9dd-a2cdba7b1c29',
  '3a657a6d-8814-4236-af6f-8a549be5123e'],
 'embeddings': array([[-2.07734108e-03, -2.16356432e-03,  2.67739072e-02, ...,
         -1.70658119e-02, -3.68524459e-03,  1.35402400e-02],
        [-2.70681921e-03, -5.47363961e-05,  2.82030534e-02, ...,
         -1.50933377e-02,  5.93815418e-03, -1.16601437e-02],
        [ 7.83727213e-04, -4.76402277e-03,  1.23708826e-02, ...,
         -1.72387529e-02,  7.74397107e-04,  2.93532619e-03],
        [-2.71391869e-02,  8.87096301e-03,  2.69362777e-02, ...,
         -2.58583203e-02,  9.02314577e-03, -1.99992992e-02],
        [-1.81433018e-02,  1.27527108e-02,  3.47932205e-02, ...,
         -3.03654689e-02, -5.85236121e-03,  5.21374308e-03]],
       shape=(5, 1536)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his a

In [ ]:
# delete document
vector_store.delete(ids=['4fbadedb-e4e1-4da1-87c3-800b8ad7ca3e'])

In [24]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['4fbadedb-e4e1-4da1-87c3-800b8ad7ca3e',
  'c21aa93f-2f36-4bb6-a565-e2818f6a2bce',
  'c8bca5e8-3caf-43f6-8557-1a75623a9dc8',
  'c83ffd12-78ea-4d67-a9dd-a2cdba7b1c29',
  '3a657a6d-8814-4236-af6f-8a549be5123e'],
 'embeddings': array([[-2.07734108e-03, -2.16356432e-03,  2.67739072e-02, ...,
         -1.70658119e-02, -3.68524459e-03,  1.35402400e-02],
        [-2.70681921e-03, -5.47363961e-05,  2.82030534e-02, ...,
         -1.50933377e-02,  5.93815418e-03, -1.16601437e-02],
        [ 7.83727213e-04, -4.76402277e-03,  1.23708826e-02, ...,
         -1.72387529e-02,  7.74397107e-04,  2.93532619e-03],
        [-2.71391869e-02,  8.87096301e-03,  2.69362777e-02, ...,
         -2.58583203e-02,  9.02314577e-03, -1.99992992e-02],
        [-1.81433018e-02,  1.27527108e-02,  3.47932205e-02, ...,
         -3.03654689e-02, -5.85236121e-03,  5.21374308e-03]],
       shape=(5, 1536)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his a